# Benchmarking pour une seule prédiction


@Author Colin de Seroux


## <span style="color:lightblue">Installation des dépendances</span>


In [ ]:
%pip install -r requirements.txt

## <span style="color:lightblue">Importation des dépendances</span>


In [2]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from utils import extract_annotations_from_file, extract_predictions_from_file

## <span style="color:lightblue">Code principale</span>


### <span style="color:lightgreen">Environnement</span>


In [3]:
ANNOTATIONS_FILE_PATH = "../data/dataset4/test/result.json"
PREDICTIONS_FILE_PATH = "../results/predictions/predictions_RTX 3080 TI laptop 95W_yolo11n_trained.pt.json"

### <span style="color:lightgreen">Récupération des données</span>


In [ ]:
y_true, class_names = extract_annotations_from_file(ANNOTATIONS_FILE_PATH)
coco = COCO(ANNOTATIONS_FILE_PATH)
y_pred, y_scores = extract_predictions_from_file(coco, PREDICTIONS_FILE_PATH)
coco_annotations = COCO(ANNOTATIONS_FILE_PATH)
coco_predictions = coco_annotations.loadRes(PREDICTIONS_FILE_PATH)

### <span style="color:lightgreen">mAP</span>


Initialiser l'évaluation


In [5]:
coco_eval = COCOeval(coco_annotations, coco_predictions, iouType="bbox")

Calculer le mAP


In [ ]:
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

### <span style="color:lightgreen">F1 Score</span>


In [ ]:
precision, recall, scores, iou_lookup = evaluate_metrics(coco_eval)
display_metrics(precision, recall, iou_lookup, None)

# TODO des corrections restent à faire
# for cat in coco_annotations.loadCats(coco_annotations.getCatIds()):
#         coco_eval.params.catIds = [cat["id"]]
#         precision, recall, scores, iou_lookup = evaluate_metrics(coco_eval)
#         display_metrics(precision, recall, iou_lookup, class_name=cat["name"])

### <span style="color:lightgreen">Matrice de confusion</span>


Préparation des données pour la matrice de confusion.


In [8]:
y_true_flat, y_pred_flat, classes = prepare_confusion_matrix(y_true, y_pred, class_names)

Ajout d'une classe 'background' lorsque rien n'est détecté.


In [9]:
class_names.append("background")

Calcul de la matrice de confusion.


In [10]:
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=classes)

Afficher la matrice de confusion.


In [ ]:
plot_confusion_matrix(cm, "d", class_names, "Matrice de confusion")

Afficher la matrice de confusion normalisée.


In [ ]:
cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

plot_confusion_matrix(cm_normalized, ".2f", class_names, "Matrice de confusion normalisée")

## <span style="color:lightblue">Sources</span>


- https://gist.github.com/shivamsnaik/c5c5e99c00819d2167317b1e56871187
- https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.f1_score.html
- https://kobia.fr/classification-metrics-f1-score
